# 整数ナップサック問題

同じ品物を何個入れても良いという問題設定のナップサック問題

$$
\max \sum_{i=1}^n v_i x_i \\
\mathrm{subject \ to \ } \sum_{i=1}^n s_i x_i \le b \\
x_i \in \{0,1,2,\ldots\},  \quad i = 1,\ldots, n
$$

## 動的計画法による解法

ナップサック容量 $b$ を $\theta$ に取り替えた問題の最適解を $f(\theta)$ とおき，$\theta=0,1,2,\ldots,b$ で解いていく

$$
f(0) = 0, \\
f(\theta) = -\infty \quad (\theta<0)
$$
と定義して，$\theta>0$ に対して
$$
f(\theta) = \max \{ 0, \max_{i=1,\ldots,n} f(\theta-s_i) + v_i \}
$$
と計算していく

0-1 ナップサック (同じ品物を二度選べない問題設定) の場合よりDPテーブルは小さい(サイズ $b$)が，上式の計算に $O(n)$ かかるので，計算時間は0-1の場合と同じで $O(nb)$

In [1]:
def ikpdp(s, v, b):
    memo = {} # 部分問題の最適解の目的関数値とそのステップで入れたアイテム(入れてなければ-1)のタプルを保持する

    def f(b):
        # 既に一度f(b)が計算されていれば，辞書memoから読み出す．そうでなければ再帰的に計算する
        if b == 0:
            return 0, -1
        if b < 0:
            return -999999, -1
        if b in memo:
            return memo[b]
        else:
            max_value = 0
            prev = -1
            for i, size in enumerate(s):
                if f(b - size)[0] + v[i] > max_value:
                    max_value = f(b - size)[0] + v[i]
                    prev = i
            memo[b] = max_value, prev
        return memo[b]

    opt_val, prev = f(b)
    x = [0 for i in range(len(s))]
    # memo から入れた品物を順番に読み出す
    while True:
        val, prev = memo[b]
        if prev == -1: break # テキストの実装のバグ修正で1行追加
        x[prev] += 1
        b -= s[prev]
        if b <= 0:
            break

    return opt_val, x

In [2]:
s = [2, 3, 4, 5]
v = [16, 19, 23, 28]
b = 7
opt_val, x = ikpdp(s, v, b)
print("Opt. value=", opt_val)
print("Sol.=", x)

Opt. value= 51
Sol.= [2, 1, 0, 0]


In [3]:
# 挙動確認: f の値は階段上に増えていく
s = [2]
v = [16]
for b in range(1,7):
    opt_val, x = ikpdp(s, v, b)
    print("Opt. value=", opt_val)
    print("Sol.=", x)

Opt. value= 0
Sol.= [0]
Opt. value= 16
Sol.= [1]
Opt. value= 16
Sol.= [1]
Opt. value= 32
Sol.= [2]
Opt. value= 32
Sol.= [2]
Opt. value= 48
Sol.= [3]


# 0-1 ナップサック問題

各品物が1個ずつしかない問題設定のナップサック問題

$$
\max \sum_{i=1}^n v_i x_i \\
\mathrm{subject \ to \ } \sum_{i=1}^n s_i x_i \le b \\
x_i \in \{0,1\},  \quad i = 1,\ldots, n
$$

## 動的計画法による解法

こちらのDPの方がよく教科書で見る気がする．

$n$ を $k=0, 1,\ldots,n$ に取り替え，ナップサック容量 $b$ を $\theta$ に取り替えた問題の最適解を $f(k,\theta)$ とおく

$$
f(0, \theta) = 0, \\
f(k, \theta) = -\infty \quad (\theta<0)
$$
と定義して，以下に従って計算していく
$$
f(k, \theta) = \max \big\{ f(k-1, \theta), \ f(k-1,\theta-s_k) + v_k \big\}
$$

- 注: [テキスト](https://scmopt.github.io/opt100/77mkp.html)と微妙に異なるが，この書き方の方が整数ナップサック問題との一貫性がある気がするのでこのように書いた．
    - この式に従った更新式を下に `kpdp_2` として実装してみたが，$i=1,\ldots,n$ の表記を使う都合上 $v,s,x$ へのアクセスを全て $k-1$ に書き直す必要がある．テキストでの実装はこれを避けるためか．

In [4]:
# テキストの実装
def kpdp(s, v, b):
    memo = {}

    def f(k, b):
        if b < 0:
            return -9999, 0
        if k == 0:
            if b >= s[0]:
                memo[0, b] = v[0], 1
                return memo[0, b]
            else:
                return 0, 0
        if (k, b) in memo:
            return memo[k, b]
        else:
            if f(k - 1, b)[0] < f(k - 1, b - s[k])[0] + v[k]:
                max_value = f(k - 1, b - s[k])[0] + v[k]
                sol = 1
            else:
                max_value = f(k - 1, b)[0]
                sol = 0
            memo[k, b] = max_value, sol
            return memo[k, b]

    opt_val, sol = f(len(s) - 1, b)

    x = [0 for i in range(len(s))]
    for k in range(len(s) - 1, -1, -1):
        val, sol = memo[k, b]
        if sol == 1:
            x[k] += 1
        b -= s[k] * sol
        if b <= 0:
            break

    return opt_val, x

In [5]:
# 変更した実装
def kpdp_2(s, v, b):
    memo = {}

    def f(k, b):
        if b < 0:
            return -9999, 0
        if k == 0:
            return 0, 0
        if (k, b) in memo:
            return memo[k, b]
        else:
            if f(k - 1, b)[0] < f(k - 1, b - s[k-1])[0] + v[k-1]:
                max_value = f(k - 1, b - s[k-1])[0] + v[k-1]
                sol = 1
            else:
                max_value = f(k - 1, b)[0]
                sol = 0
            memo[k, b] = max_value, sol
            return memo[k, b]

    opt_val, sol = f(len(s), b)

    x = [0 for i in range(len(s))]
    for k in range(len(s), 0, -1):
        val, sol = memo[k, b]
        if sol == 1:
            x[k-1] += 1
        b -= s[k-1] * sol
        if b <= 0:
            break

    return opt_val, x

In [6]:
s = [2, 3, 4, 5]
v = [16, 19, 23, 28]
b = 7
opt_val, x = kpdp(s, v, b)
print("Opt. value=", opt_val)
print("Sol.=", x)

opt_val, x = kpdp_2(s, v, b)
print("Opt. value=", opt_val)
print("Sol.=", x)

Opt. value= 44
Sol.= [1, 0, 0, 1]
Opt. value= 44
Sol.= [1, 0, 0, 1]


In [7]:
s = [2] * 5
v = [16] * 5
b = 7
opt_val, x = kpdp(s, v, b)
print("Opt. value=", opt_val)
print("Sol.=", x)

opt_val, x = kpdp_2(s, v, b)
print("Opt. value=", opt_val)
print("Sol.=", x)

Opt. value= 48
Sol.= [1, 1, 1, 0, 0]
Opt. value= 48
Sol.= [1, 1, 1, 0, 0]
